<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Auto-Complete-Beginner/blob/main/NLP_Auto_Complete_Beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!pip install -q -U trax

import trax

# import trax.fastmath.numpy as np

from trax import layers as tl

In [3]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%203/en_US.twitter.txt

--2022-04-20 08:09:18--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%203/en_US.twitter.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3341555 (3.2M) [text/plain]
Saving to: ‘en_US.twitter.txt.3’

en_US.twitter.txt.3 100%[===================>]   3.19M  --.-KB/s    in 0.07s   

2022-04-20 08:09:18 (47.6 MB/s) - ‘en_US.twitter.txt.3’ saved [3341555/3341555]



In [4]:
with open('en_US.twitter.txt', 'r') as f:
  data = f.read()

In [5]:
print(len(data), type(data))

3335477 <class 'str'>


In [6]:
display(data[0:400])

"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)\nWords from a complete stranger! Made my birthday eve"

In [7]:
display(data[-400:])

"tplace jerks off aristocrats until they finally die in the midst of self-love.\n's icon is life.\nwe just had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

In [8]:
def split2Sentences(data):
  
  sentences = data.split('\n')
  sentences = [s.strip() for s in sentences]
  sentences = [s for s in sentences if len(s)>0]
  return sentences

In [9]:
sentences = split2Sentences(data)
sentences[0:10]

['How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.',
 "When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.",
 "they've decided its more fun if I don't.",
 'So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)',
 'Words from a complete stranger! Made my birthday even better :)',
 'First Cubs game ever! Wrigley field is gorgeous. This is perfect. Go Cubs Go!',
 'i no! i get another day off from skool due to the wonderful snow (: and THIS wakes me up...damn thing',
 "I'm coo... Jus at work hella tired r u ever in cali",
 'The new sundrop commercial ...hehe love at first sight',
 'we need to reconnect THIS WEEK']

In [10]:
nltk.word_tokenize(sentences[2])

['they', "'ve", 'decided', 'its', 'more', 'fun', 'if', 'I', 'do', "n't", '.']

In [11]:
tokenize_sentences =lambda sentences: [nltk.word_tokenize(s.lower()) for s in sentences]

In [12]:
tokenized_sentences = tokenize_sentences(sentences)

In [13]:
tokenized_sentences[0]

['how',
 'are',
 'you',
 '?',
 'btw',
 'thanks',
 'for',
 'the',
 'rt',
 '.',
 'you',
 'gon',
 'na',
 'be',
 'in',
 'dc',
 'anytime',
 'soon',
 '?',
 'love',
 'to',
 'see',
 'you',
 '.',
 'been',
 'way',
 ',',
 'way',
 'too',
 'long',
 '.']

In [14]:
data_tokenizer = lambda data: tokenize_sentences(split2Sentences(data))

In [15]:
tokenized_sentences = data_tokenizer(data)

In [16]:
np.random.shuffle(tokenized_sentences)
train_size = int(len(tokenized_sentences)*0.8)
train_data, test_data = tokenized_sentences[:train_size], tokenized_sentences[train_size:]

In [17]:
print(len(train_data), len(test_data))

38368 9593


In [18]:
def word_counter(tokenized_sentences):
  word_dic = {}
  for sentence in tokenized_sentences:
    for token in sentence:
      if token not in word_dic:
        word_dic[token] = 1
      else:
        word_dic[token] += 1
  return word_dic
  # return collections.Counter(tokenized_sentences)

In [19]:
word_c = word_counter(tokenized_sentences)

for i, w in enumerate(word_c):
  if i == 20:
    break

  print(w)

thank
you
for
the
best
night
ever
with
word
is
chris
hardwick
may
be
looking
at
a
guest
set
tonight


In [20]:
def words_with_nplus_frequncy(tokenize_sentences, count_threshold):
  closed_vocab = []
  word_count = word_counter(tokenized_sentences)
  for k, v in word_count.items():
    if v >= count_threshold:
      closed_vocab.append(k)
  return closed_vocab

In [21]:
closed_vocab = words_with_nplus_frequncy(tokenized_sentences,2)

In [22]:
closed_vocab[: 20]

['thank',
 'you',
 'for',
 'the',
 'best',
 'night',
 'ever',
 'with',
 'word',
 'is',
 'chris',
 'may',
 'be',
 'looking',
 'at',
 'a',
 'guest',
 'set',
 'tonight',
 'and']

In [64]:
def oov2Unk(tokenized_data, vocab, unknown_token='<unk>'):
  vocab = set(vocab)

  replaced_tokenized_sentences = []
  for i, sentence in enumerate(tokenized_data):

    replace_sentences = []
    for token in sentence:

      temp_token = token if token in vocab else unknown_token
      replace_sentences.append(temp_token)
    replaced_tokenized_sentences.append(replace_sentences)
  return replaced_tokenized_sentences

In [65]:
def preprocess_data(train_data, test_data, count_threshol=2):
  
  vocab = words_with_nplus_frequncy(train_data, count_threshol)
  train_data_replaced = oov2Unk(train_data, vocab)
  test_data_replaced = oov2Unk(test_data, vocab)
  return train_data_replaced, test_data_replaced, vocab

In [66]:
train_data_replaced, test_data_replaced, vocab = preprocess_data(train_data, test_data)

In [70]:
train_data_replaced[20]

['been',
 'reading',
 'your',
 'tweets',
 'and',
 'i',
 'like',
 'your',
 'style',
 '.',
 'thanks',
 'for',
 'being',
 'one',
 'of',
 'the',
 'only',
 'followers',
 'to',
 'not',
 'use',
 'an',
 '<unk>',
 '!']

In [71]:
test_data_replaced[20]

['if',
 'for',
 'some',
 'reason',
 'our',
 '<unk>',
 'link',
 'dosent',
 'work',
 'for',
 'you',
 ',',
 'you',
 'can',
 'download',
 'our',
 'demos',
 'on',
 'the',
 '``',
 '<unk>',
 "''",
 'tab',
 'on',
 'our',
 'page',
 '!']

In [73]:
def creat_n_grams(data, n, start_token='<s>', end_token='<e>'):
  n_grams = {}
  for sentence in data:
    sentence = [start_token]*n + sentence + [end_token]
    sentence = tuple(sentence)
    m = len(sentence) if n==1 else len(sentence)-1

    for i in range(m):
      n_gram = sentence[i: i+n]
      if n_gram in n_grams.keys():
        n_grams[n_gram] += 1
      else:
        n_grams[n_gram] = 1
  return n_grams

In [82]:
for i in creat_n_grams(test_data_replaced, 1).items():
  print(i)
  break

(('<s>',), 9593)


In [83]:
creat_n_grams(test_data_replaced, 1)

{('<s>',): 9593,
 ('eric',): 10,
 ("'s",): 1177,
 ('poem',): 4,
 ('is',): 1508,
 ('``',): 546,
 ('diesel',): 4,
 (',',): 3035,
 ("''",): 552,
 ('and',): 1776,
 ('the',): 3808,
 ('february',): 4,
 ('2011',): 16,
 ('<unk>',): 5316,
 ('.',): 6028,
 ('represented',): 1,
 ('in',): 1558,
 ('january',): 5,
 ('with',): 720,
 ('new',): 269,
 ('year',): 107,
 ('<e>',): 9593,
 ('when',): 333,
 ('people',): 227,
 ('touch',): 9,
 ('my',): 1206,
 ('feet',): 11,
 ('<',): 208,
 ('#',): 1078,
 ('we',): 628,
 ('have',): 748,
 ('decided',): 8,
 ('to',): 3162,
 ('change',): 40,
 ('our',): 228,
 ('last',): 142,
 ('names',): 11,
 ('smith',): 4,
 ('you',): 2388,
 ("'re",): 252,
 ('right',): 182,
 ('there',): 256,
 ('another',): 78,
 ('label',): 3,
 ('for',): 1555,
 ('it',): 1617,
 ('but',): 517,
 ('inappropriate',): 1,
 ('fastest',): 2,
 ('way',): 141,
 ('ignore',): 8,
 ('begin',): 8,
 ('warning',): 2,
 ('24',): 8,
 ('hours',): 37,
 ('advance',): 6,
 ('about',): 346,
 ('potentially',): 1,
 ('deadly',): 1,
 (

In [88]:
for i in creat_n_grams(test_data_replaced, 2).items():
  print(i)
  break

(('<s>', '<s>'), 9593)
